In [1]:
#Watson s2t using websockets: websockets2

In [2]:
from ws4py.client.threadedclient import WebSocketClient
import base64, json, ssl, subprocess, threading, time
import io

In [3]:
class SpeechToTextClient(WebSocketClient):
    def __init__(self):
        ws_url = "wss://stream.watsonplatform.net/speech-to-text/api/v1/recognize"

        username = ""
        password = ""

        auth_string = "%s:%s" % (username, password)
        b64Val = base64.b64encode(auth_string.encode("utf-8"))
        b64Val1= b64Val.decode("utf-8")
        b64Val1 = auth_string
        self.base64string = base64.encodestring(b64Val1).replace("\n", "")

        self.listening = False

        try:
            WebSocketClient.__init__(self, ws_url,
                headers=[("Authorization", "Basic %s" % self.base64string)])
            self.connect()
        except: print "Failed to open WebSocket."

    def opened(self):
        opendata = {"action": "start", 
                    "content-type": "audio/l16;rate=44100", 
                    "interim_results": True, 
                    "continuous": True, 
                    "inactivity_timeout":5,
                   }
        self.send(json.dumps(opendata).encode('utf8'))
        self.stream_audio_thread = threading.Thread(target=self.stream_audio)
        self.stream_audio_thread.start()

    def received_message(self, message):
        message = json.loads(str(message))
        if "state" in message:
            if message["state"] == "listening":
                self.listening = True
            print "Message received: " + str(message)
        elif "results" in message:
            try:
                print message['results'][0]['alternatives'][0]['transcript'] + ' '
            except:
                print "Message received: (results error)" + str(message)
        elif "error" in message:
            print "End of Demo: Please hit <Enter> at prompt to close"
        else:
            print "Message received: " + str(message)
        
    def stream_audio(self):
        while not self.listening:
            time.sleep(0.1)
  
        command = ['ffmpeg', '-f', 'dshow', '-i', 'audio=Microphone (Realtek High Definition Audio)', 
                  '-ac', '1', '-f', 's16le', '-acodec', 'pcm_s16le', '-ar', '44100','-']

        p = subprocess.Popen(command,stdout=subprocess.PIPE)

        while self.listening:
            data = p.stdout.read(1024)
            #print 'len: ', len(data)
            if len(data) != 0:
                try:
                    self.send(bytearray(data), binary=True)
                    #self.send(str(data), binary=True)
                except ssl.SSLError as e:
                    print ("SSL error: " + str(e))
                except:
                    break
        p.kill()
    
    def myclose(self):
        self.listening = False
        self.stream_audio_thread.join()
        WebSocketClient.close(self)      
        

In [4]:
try:
    stt_client = SpeechToTextClient()
    raw_input()
finally:
    stt_client.myclose()
    print 'Thank you'

Message received: {u'state': u'listening'}
hello  
hello  
hello  
hello  
I  
how does  
I don't  
I know  
I know  
this  
this is  
this is true  
this is just a  
this is just dance  
this is just dance  
this is just danced  
this is just danced  
none  
don't know  
don't mix  
don't make some  
don't websocket  
don't websocket  
don't websocket  
I  
I  
IBM  
IBM  
IBM  
IBM where  
IBM with  
IBM with the  
IBM red lips  
IBM red lips soft  
IBM red websocket  
IBM red websocket  
IBM red websocket  
he  
here  
here is  
here is  
here is  
here is  
here is  
my  
my  
my  
my  
my ex  
my experiment  
my experiment  
my experiment  
my experiment  
my experiment  
who  
we  
we  
we I  
who it might be  
with IBM  
with IBM  
with IBM  
with IBM  
where  
website  
website  
websocket  
websocket  
Rep sockets  
Rep sockets  
End of Demo: Please hit <Enter> at prompt to close

Thank you
